<a href="https://colab.research.google.com/github/sblHead/Supplementary-code/blob/main/S_NN_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from google.colab import drive
import csv
import sklearn


drive.mount('/content/drive/')    # if importing data from google drive

Mounted at /content/drive/


In [ ]:
input_shape = 6
Xdata = []

with open('', newline = '\n') as csvfile:            # Read in material data file
  data = csv.reader(csvfile, delimiter = ',')
  for row in data:
    row = np.array(row)
    row =  row.astype(np.float)
    Xdata.append(row)
    
Xdata = np.delete(Xdata,slice(0,len(Xdata),2),axis = 0)

Ydata = []
with open('', newline = '\n') as csvfile:       # Read in spectral response data file
  data = csv.reader(csvfile, delimiter = ',')
  for row in data:
    row = np.array(row)
    row =  row.astype(np.float)
    Ydata.append(row)
Ydata = np.transpose(Ydata)
Ydata = np.delete(Ydata,slice(0,len(Ydata),2),axis = 0)



print(np.shape(Xdata))       # ensure that each row is a new sample, i.e. shape = (num samples, len sample)
print(np.shape(Ydata))

In [ ]:
epoch_num = 40


XTrain = []
YTrain = []
XTest = []
YTest = []
XNew = []
YNew = []

XShuff, YShuff = sklearn.utils.shuffle(Xdata, Ydata)

XTrain = XShuff[:18000]
YTrain = YShuff[:18000]
XTest = XShuff[18000:21000]
YTest = YShuff[18000:21000]
XNew = XShuff[21000:]
YNew = YShuff[21000:]

XTrain = np.array(XTrain)
YTrain = np.array(YTrain)
XTest = np.array(XTest)
YTest = np.array(YTest)

In [ ]:

input_tensor = keras.Input(shape = (input_shape))
input_tensor= keras.layers.BatchNormalization(axis=-1)(input_tensor)

x = keras.layers.Dense(500, activation = None)(input_tensor)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
x = keras.layers.Dropout(0.02)(x)


x = keras.layers.Dense(500, activation = None)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)
x = keras.layers.Dropout(0.01)(x)


x = keras.layers.Dense(500, activation = None)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)



x = keras.layers.Dense(1000, activation = None)(x)
x = keras.layers.LeakyReLU(alpha=0.2)(x)


output_tensor = keras.layers.Dense(81, activation = 'relu')(x)

model = keras.models.Model(input_tensor, output_tensor)

model.compile(loss = keras.losses.MeanSquaredError(),
              optimizer = 'Adam',
              metrics = ['mean_absolute_error'])


callback_list = [keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=4, min_lr=1e-6), 
                 keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)]

history = model.fit(XTrain, YTrain, epochs = epoch_num, callbacks = callback_list, validation_data = (XTest, YTest),verbose =1)

#model.summary()
#print(model.evaluate(XNew,YNew,verbose = 0))

In [ ]:
print(model.evaluate(XNew,YNew,verbose = 0))

In [ ]:
model.save('')      # Save model to desire location

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
loss_train = history.history['mean_absolute_error']         # Produces a learning curve graph showing training and validation curves
loss_val = history.history['val_mean_absolute_error']
epochs = range(1,40)
plt.figure(figsize = (3.5,3.5),dpi=1200)
plt.plot(epochs, loss_train[1:], 'g', label='Training MSE')
plt.plot(epochs, loss_val[1:], 'b', label='Validation MSE')
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Epochs', fontsize=16)
plt.ylabel('Mean Absolute Error',fontsize=16)
plt.show()

In [ ]:
num = ##                                                    # produces a graph comparing target and predicted spectra     
Y_hat = model.predict(XNew[num].reshape(-1,6))
Y_acc = YNew[num].reshape(81, -1)
Y_hat = np.transpose(Y_hat)
spectrum_wav = np.arange(380, 781, 5)
# plt.figure(figsize = (3.5,3.5),dpi=1200)
plt.plot(spectrum_wav, Y_hat, 'b')
plt.plot(spectrum_wav, Y_acc, 'r')
plt.ylim([0, 1])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.xlabel('Wavelength, nm', fontsize=16)
plt.ylabel('Reflectance coefficient',fontsize=16)
plt.show()